In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import nltk
import re
import pickle

### Loading Data

In [2]:
news_total = pd.read_csv("/kaggle/input/arabic-text-summarization/summarizdataset.csv")

In [3]:
news = news_total[["text", "summarizer"]]

In [4]:
news["text"][2]

'\nاحتضن جناح تونس في القرية الدولية للأفلام بمدينة "كان" الفرنسية التي تستضيف الدورة السبعين لمهرجان كان السينمائي، لقاء جمع المدير العام للمركز الوطني للسينما والصورة، فتحي الخراط ومسؤولين بالمركز الفرنسي للسينما لتقديم إتفاقية بعث صندوق إنتاج مشترك تونسي فرنسي لتمويل الأفلام.\nوأوضح الخراط في تصريح أنه سيتم الشروع في تفعيل هذه الإتفاقية التي وقّعت في شهر فيفري الفارط بمناسبة زيارة وزيرة الثقافة والاتصال الفرنسية السابقة إلى تونس، مضيفا أن "صندوق الإنتاج المشترك، جاء ليوفر مصدر تمويل جديد للسينمائيين التونسيين"، وفق تأكيده. التعاون من "جحا" وقال مدير المركز الوطني للسينما والصورة إن "التعاون الثنائي بين البلدين في المجال السينمائي يعود إلى سنة 1957 لينطلق مع الفيلم التونسي، "جحا" الذي أخرجه السينمائي الفرنسي، جان باراتييه، موضحا أن "هذا الشريط يعد أول عمل سينمائي أنجز في تونس المستقلة"، على حد تعبيره. واستعرض الجانبان التونسي الفرنسي بالمناسبة، أهم البنود التي جاءت في الإتفاقية والتي تحدد إلتزامات كل طرف، فضلا عن تركيبة اللجنة التي ستنظر في المشاريع المعروضة عليها والشروط الواجب توفر

In [5]:
news.head()

,text,summarizer
0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...","\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون..."
2,\nاحتضن جناح تونس في القرية الدولية للأفلام بم...,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...
3,\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...
4,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...


In [6]:
news.shape

(8378, 2)

In [7]:
document = news['text']
summary = news['summarizer']

In [8]:
document[30], summary[30]

('\nقال إبراهيم لطيف مدير الدورة 27 لأيام قرطاج السينمائية في تصريح للجوهرة أف أم اليوم الاثنين 14 نوفمبر 2016 إنه قد أعلن عن استقالته منذ يوم الجمعة الماضي أي قبل صدور قرار وزارة الشؤون الثقافية المتعلق بوضع حد لتكليفه.\nوأضاف لطيف أنه تفاجأ بقرار\xa0إنهاء تكليفه مؤكدا أنه لم يتم إعلامه بذلك. وقال لطيف إنه مستعد للمحاسبة وعلى وزير الشؤون الثقافية تحمل مسؤوليته وفق تعبيره.\xa0\xa0 وكانت وزارة الشؤون الثقافية قد دعت في بلاغ أصدرته اليوم الهيئة المديرة للدورة 27 لأيام قرطاج السينمائية إلى تقديم التقريرين المالي والأدبي للدورة المذكورة في أقرب االآجال.\n',
 'وكانت وزارة الشؤون الثقافية قد دعت في بلاغ أصدرته اليوم الهيئة المديرة للدورة 27 لأيام قرطاج السينمائية إلى تقديم التقريرين المالي والأدبي للدورة المذكورة في أقرب االآجال. وقال لطيف إنه مستعد للمحاسبة وعلى وزير الشؤون الثقافية تحمل مسؤوليته وفق تعبيره. وأضاف لطيف أنه تفاجأ بقرار\xa0إنهاء تكليفه مؤكدا أنه لم يتم إعلامه بذلك.')

### Preprocessing

In [9]:
# for decoder sequence
summary = summary.apply(lambda x: '<go> ' + x + ' <stop>')
summary.head()

0    <go> \nأشرف رئيس الجمهورية الباجي قايد السبسي ...
1    <go> \nتحصل كتاب "المصحف وقراءاته" الذي ألفه ب...
2    <go> تونس حاضرة من جهة أخرى ستكون تونس حاضرة ف...
3    <go> واستأجرت صاحبة المشروع المحامية والكاتبة ...
4    <go> \nنعت وزارة الشّؤون الثّقافيّة المنشد الص...
Name: summarizer, dtype: object

#### Tokenizing the texts into integer tokens

In [10]:
# since < and > from default tokens cannot be removed
filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n\xa0'
oov_token = '<unk>'

In [11]:
counter = 0
def delete_links(input_text):
    pettern  = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
    out_text = re.sub(pettern, ' ', input_text)
    return out_text

def delete_repeated_characters(input_text):
    pattern  = r'(.)\1{2,}'
    out_text = re.sub(pattern, r"\1\1", input_text)
    return out_text

def remove_extra_spaces(input_text):
    replace = ' +'
    out_text = re.sub(replace, " ", input_text)
    words = nltk.word_tokenize(out_text)
    words = [word for word in words if word.isalpha()]
    out_text = ' '.join(words)
    return out_text

def replace_letters(input_text):
    replace = {"أ": "ا","ة": "ه","إ": "ا","آ": "ا","": ""}
    replace = dict((re.escape(k), v) for k, v in replace.items())
    pattern = re.compile("|".join(replace.keys()))
    out_text = pattern.sub(lambda m: replace[re.escape(m.group(0))], input_text)
    return out_text

def clean_text(input_text):
    replace = r'[^\u0621-\u064A\u0660-\u0669\u06F0-\u06F90-9]'
    out_text = re.sub(replace, " ", input_text)
    #words = nltk.word_tokenize(out_text)
    #words = [word for word in words if word.isalpha()]
    #out_text = ' '.join(words)
    return out_text

def remove_vowelization(input_text):
    vowelization = re.compile(""" ّ|َ|ً|ُ|ٌ|ِ|ٍ|ْ|ـ""", re.VERBOSE)
    out_text = re.sub(vowelization, '', input_text)
    return out_text

def delete_stopwords(input_text):
    stop_words = set(nltk.corpus.stopwords.words("arabic") + nltk.corpus.stopwords.words("english"))
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(input_text)
    wnl = nltk.WordNetLemmatizer()
    lemmatizedTokens =[wnl.lemmatize(t) for t in tokens]
    out_text = [w for w in lemmatizedTokens if not w in stop_words]
    out_text = ' '.join(out_text)
    return out_text

def stem_text(input_text):
    st = ISRIStemmer()
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(input_text)
    out_text = [st.stem(w) for w in tokens]
    out_text = ' '.join(out_text)
    return out_text


def text_prepare(input_text, ar_text):
    global counter
    counter +=1

    out_text = delete_links(input_text)
    out_text = delete_repeated_characters(out_text)
    out_text = delete_stopwords(input_text)
    out_text = clean_text(input_text)
    out_text = remove_extra_spaces(out_text)
    if(counter%100==0):
        print(counter,'\n',out_text)
    return out_text

In [ ]:
xl_sum = []

In [13]:
xl_sum  = news
xl_sum.head()

,text,summarizer
0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...","\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون..."
2,\nاحتضن جناح تونس في القرية الدولية للأفلام بم...,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...
3,\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...
4,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...


In [18]:
xl_sum['paragraph'] = xl_sum['text'].apply(text_prepare, args=(True,))
xl_sum['summary'] = xl_sum['summarizer'].apply(text_prepare, args=(True,))

LookupError: 
**********************************************************************
  Resource [93momw-1.4[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('omw-1.4')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/omw-1.4[0m

  Searched in:
    - '/root/nltk_data'
    - '/opt/conda/nltk_data'
    - '/opt/conda/share/nltk_data'
    - '/opt/conda/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [12]:
document_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)
summary_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters, oov_token=oov_token)
document_tokenizer.fit_on_texts(document)
summary_tokenizer.fit_on_texts(summary)

In [13]:
inputs = document_tokenizer.texts_to_sequences(document)
targets = summary_tokenizer.texts_to_sequences(summary)

In [14]:
summary_tokenizer.texts_to_sequences([" كل ابن أُنثى وإن طالت سلامتُهُ بوماً على آلةٍ حدباء محمول "])

[[56, 1832, 1, 1612, 3845, 1, 1, 6, 1, 1, 8323]]

In [15]:
summary_tokenizer.sequences_to_texts([[56, 1813, 1, 1690, 3818, 1, 1, 6, 1, 1, 8275]])

['كل بقابس <unk> الخطوة يلتقي <unk> <unk> على <unk> <unk> بنظام']

In [16]:
encoder_vocab_size = len(document_tokenizer.word_index) + 1
decoder_vocab_size = len(summary_tokenizer.word_index) + 1

# vocab_size
encoder_vocab_size, decoder_vocab_size

(92110, 49459)

#### Obtaining insights on lengths for defining maxlen

In [17]:
document_lengths = pd.Series([len(x) for x in document])
summary_lengths = pd.Series([len(x) for x in summary])

In [18]:
document_lengths.describe()

count    8378.000000
mean      663.660062
std       491.505431
min        82.000000
25%       366.000000
50%       524.500000
75%       793.000000
max      9854.000000
dtype: float64

In [19]:
summary_lengths.describe()

count    8378.000000
mean      261.807591
std       117.474832
min        22.000000
25%       160.000000
50%       257.000000
75%       351.000000
max       596.000000
dtype: float64

In [20]:
# maxlen
# taking values > and round figured to 75th percentile
# at the same time not leaving high variance
encoder_maxlen = 400
decoder_maxlen = 75

#### Padding/Truncating sequences for identical sequence lengths

In [21]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=encoder_maxlen, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=decoder_maxlen, padding='post', truncating='post')

### Creating dataset pipeline

In [22]:
inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)

In [23]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [24]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

### Positional Encoding for adding notion of position among words as unlike RNN this is non-directional

In [25]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


### Masking

- Padding mask for masking "pad" sequences
- Lookahead mask for masking future words from contributing in prediction of current words in self attention

In [26]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

### Building the Model

#### Scaled Dot Product

In [27]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, v)
    return output, attention_weights

#### Multi-Headed Attention

In [28]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)
            
        return output, attention_weights

### Feed Forward Network

In [29]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

#### Fundamental Unit of Transformer encoder

In [30]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2


#### Fundamental Unit of Transformer decoder

In [31]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2



#### Encoder consisting of multiple EncoderLayer(s)

In [32]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
    
        return x


#### Decoder consisting of multiple DecoderLayer(s)

In [33]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        return x, attention_weights



#### Finally, the Transformer

In [34]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)

        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights


### Training

In [35]:
# hyper-params
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
EPOCHS = 100

#### Adam optimizer with custom learning rate scheduling

In [36]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


#### Defining losses and other metrics 

In [37]:
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [38]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)
train_loss = tf.keras.metrics.Mean(name='train_loss')

#### Transformer

In [39]:
transformer = Transformer(
    num_layers, 
    d_model, 
    num_heads, 
    dff,
    encoder_vocab_size, 
    decoder_vocab_size, 
    pe_input=encoder_vocab_size, 
    pe_target=decoder_vocab_size,
)

#### Masks

In [40]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
    return enc_padding_mask, combined_mask, dec_padding_mask


#### Checkpoints

In [41]:
checkpoint_path = "checkpoints"

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

#### Training steps

In [42]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp, tar_inp, 
            True, 
            enc_padding_mask, 
            combined_mask, 
            dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    return predictions

In [87]:
# # import plotly.graph_objects as go
# from tqdm import tqdm
# import time

# # Initialize lists to store training and validation/test loss
# train_losses = []
# val_losses = []

# # Loop over epochs
# for epoch in tqdm(range(EPOCHS)):
#     start = time.time()

#     train_loss.reset_states()
  
#     for (batch, (inp, tar)) in enumerate(dataset):
#         train_step(inp, tar)
    
#         # 55k samples
#         # we display 3 batch results -- 0th, middle and last one (approx)
#         # 55k / 64 ~ 858; 858 / 2 = 429
#         if batch % 429 == 0:
#             print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))

#     # Calculate and append training loss
#     train_losses.append(train_loss.result())

#     if (epoch + 1) % 5 == 0:
#         ckpt_save_path = ckpt_manager.save()
#         print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    
#     print ('Epoch {} Loss {:.4f}'.format(epoch + 1, train_loss.result()))

#     # Simulate validation/test loss calculation
#     val_loss = ...  # calculate validation/test loss
#     val_losses.append(val_loss)

#     print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))



In [48]:
# # !pip install rouge
# import rouge
# from rouge import rouge_score
# # from rouge_score import rouge_scorer
# from nltk.translate.bleu_score import corpus_bleu
# # Initialize Rouge scorer
# rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=True)

# # Initialize BLEU scorer
# def compute_bleu(reference, hypothesis):
#     return corpus_bleu([[r.split()] for r in reference], [h.split() for h in hypothesis])


NameError: name 'rouge_scorer' is not defined

In [43]:
from tqdm import tqdm 
import time

# Initialize Rouge and BLEU calculators
# rouge_calculator = RougeCalculator()
# bleu_calculator = BleuCalculator()

# Initialize metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

for epoch in tqdm(range(EPOCHS)):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
  
    for (batch, (inp, tar)) in enumerate(dataset):
        predictions = train_step(inp, tar)
    
        # Update accuracy
        train_accuracy(tar[:, 1:], predictions)

        # 55k samples
        # we display 3 batch results -- 0th, middle and last one (approx)
        # 55k / 64 ~ 858; 858 / 2 = 429
        if batch % 429 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))
      
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    
    print ('Epoch {} Loss: {:.4f}'.format(epoch + 1, train_loss.result()))
    print ('Epoch {} Accuracy: {:.4f}'.format(epoch + 1, train_accuracy.result()))

#     # Calculate Rouge and BLEU scores for training data
#     train_rouge_scores = rouge_calculator.compute_scores(train_reference, train_predictions)
#     train_bleu_score = bleu_calculator.compute_bleu(train_reference, train_predictions)

#     print ('Train Rouge Scores:', train_rouge_scores)
#     print ('Train BLEU Score:', train_bleu_score)

#     # Calculate Rouge and BLEU scores for testing data
#     test_rouge_scores = rouge_calculator.compute_scores(test_reference, test_predictions)
#     test_bleu_score = bleu_calculator.compute_bleu(test_reference, test_predictions)

#     print ('Test Rouge Scores:', test_rouge_scores)
#     print ('Test BLEU Score:', test_bleu_score)

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1 Batch 0 Loss 10.8121


  1%|          | 1/100 [01:16<2:05:40, 76.16s/it]

Epoch 1 Loss: 10.7137
Epoch 1 Accuracy: 0.0102
Time taken for 1 epoch: 76.16401386260986 secs

Epoch 2 Batch 0 Loss 10.5221


  2%|▏         | 2/100 [02:20<1:53:14, 69.33s/it]

...

Epoch 98 Loss: 0.0434
Epoch 98 Accuracy: 0.5686
Time taken for 1 epoch: 65.26409149169922 secs

Epoch 99 Batch 0 Loss 0.0336


 99%|█████████▉| 99/100 [1:48:11<01:05, 65.33s/it]

Epoch 99 Loss: 0.0427
Epoch 99 Accuracy: 0.5687
Time taken for 1 epoch: 65.22480154037476 secs

Epoch 100 Batch 0 Loss 0.0391


100%|██████████| 100/100 [1:49:17<00:00, 65.58s/it]

Saving checkpoint for epoch 100 at checkpoints/ckpt-20
Epoch 100 Loss: 0.0418
Epoch 100 Accuracy: 0.5688
Time taken for 1 epoch: 66.2869815826416 secs



In [44]:
from tqdm import tqdm 
for epoch in tqdm(range(EPOCHS)):
    start = time.time()

    train_loss.reset_states()
  
    for (batch, (inp, tar)) in enumerate(dataset):
        train_step(inp, tar)
    
        # 55k samples
        # we display 3 batch results -- 0th, middle and last one (approx)
        # 55k / 64 ~ 858; 858 / 2 = 429
        if batch % 429 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))
      
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    
    print ('Epoch {} Loss {:.4f}'.format(epoch + 1, train_loss.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1 Batch 0 Loss 0.0336


  1%|          | 1/100 [00:58<1:36:22, 58.41s/it]

Epoch 1 Loss 0.0412
Time taken for 1 epoch: 58.40574526786804 secs

Epoch 2 Batch 0 Loss 0.0288


  2%|▏         | 2/100 [01:56<1:35:25, 58.42s/it]

...

Epoch 8 Loss 0.0351
Time taken for 1 epoch: 58.44146251678467 secs

Epoch 9 Batch 0 Loss 0.0320


  9%|▉         | 9/100 [08:46<1:28:41, 58.48s/it]

Epoch 9 Loss 0.0348
Time taken for 1 epoch: 58.42677092552185 secs

Epoch 10 Batch 0 Loss 0.0238


  9%|▉         | 9/100 [09:04<1:31:42, 60.47s/it]


KeyboardInterrupt: 

In [46]:
!pip install Rouge
import Rouge
# Create an instance of the Rouge object
rouge = Rouge()
system_summaries = model_summaries["summary"].tolist()
reference_summaries = xl_test["summary"].tolist()

# Calculate RougeL scores for the list of summaries
scores = rouge.get_scores(system_summaries, reference_summaries)

# Print the RougeL scores for each summary pair
rouge_l_score = []
for i, score in enumerate(scores):
    rouge_l_score.append(score['rouge-l']['f'])
    print("RougeL Score for Summary", i + 1, ":", score['rouge-l']['f'])

print(sum(rouge_l_score)/len(rouge_l_score))

ModuleNotFoundError: No module named 'Rouge'

In [ ]:
# Create a figure with two subplots (one for training loss, one for validation/test loss)
fig = go.Figure()

# Add training loss subplot
fig.add_trace(go.Scatter(x=list(range(len(train_losses))), y=train_losses, mode='lines', name='Train Loss'))

# Add validation/test loss subplot
fig.add_trace(go.Scatter(x=list(range(len(val_losses))), y=val_losses, mode='lines', name='Validation/Test Loss'))

# Add labels and title
fig.update_layout(xaxis_title='Epoch', yaxis_title='Loss', title='Training and Validation/Test Loss')

# Show the plot
fig.show()

### Inference

#### Predicting one word at a time at the decoder and appending it to the output; then taking the complete sequence as an input to the decoder and repeating until maxlen or stop keyword appears

In [90]:
def evaluate(input_document):
    input_document = document_tokenizer.texts_to_sequences([input_document])
    input_document = tf.keras.preprocessing.sequence.pad_sequences(input_document, maxlen=encoder_maxlen, padding='post', truncating='post')

    encoder_input = tf.expand_dims(input_document[0], 0)

    decoder_input = [summary_tokenizer.word_index["<go>"]]
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(decoder_maxlen):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, 
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )

        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == summary_tokenizer.word_index["<stop>"]:
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights

In [91]:
def summarize(input_document):
    # not considering attention weights for now, can be used to plot attention heatmaps in the future
    summarized = evaluate(input_document=input_document)[0].numpy()
    summarized = np.expand_dims(summarized[1:], 0)  # not printing <go> token
    return summary_tokenizer.sequences_to_texts(summarized)[0]  # since there is just one translated document

In [92]:
summarize(
    "US-based private equity firm General Atlantic is in talks to invest about \
    $850 million to $950 million in Reliance Industries' digital unit Jio \
    Platforms, the Bloomberg reported. Saudi Arabia's $320 billion sovereign \
    wealth fund is reportedly also exploring a potential investment in the \
    Mukesh Ambani-led company. The 'Public Investment Fund' is looking to \
    acquire a minority stake in Jio Platforms."
)

'وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف وأضاف'